In [1]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook
from simpletransformers.classification import ClassificationModel, ClassificationArgs

c:\tesla-news-classifier\env\lib\site-packages\torchaudio\extension\extension.py:13: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')
c:\tesla-news-classifier\env\lib\site-packages\torchaudio\backend\utils.py:89: UserWarning: No audio backend is available.
  warnings.warn('No audio backend is available.')


In [2]:
datapath = 'data/'
train_df = pd.read_csv(datapath + 'train.csv')
test_df = pd.read_csv(datapath + 'test.csv')

In [3]:
train_df = train_df[["title", "labels"]]
train_df.columns = ["text", "labels"]

test_df = test_df[["title", "labels"]]
test_df.columns = ["text", "labels"]

In [4]:
# model_name = "bert-base-cased"
# model_type = "bert"

model_name = "distilroberta-base"
model_type = "roberta"

model_args = ClassificationArgs()
model_args.no_cache = True
model_args.fp16 = True
model_args.reprocess_input_data = True
model_args.overwrite_output_dir = True
model_args.output_dir = f"outputs/{model_type}"
model_args.best_model_dir = f"outputs/{model_type}/best_model"
model_args.evaluate_during_training = True
#model_args.lazy_loading = True
#model_args.sliding_window = True
model_args.max_seq_length = 128
model_args.num_train_epochs = 20
model_args.train_batch_size = 64
model_args.eval_batch_size = 32

In [9]:
# Calculate the label weight distribution, small label lize give more weight
distribution = train_df.labels.value_counts(normalize=False)
print(distribution)
dist = distribution.max() / distribution
print(dist)
dist = dist.sort_index().tolist()
print(dist)


0    185
2     83
1     61
Name: labels, dtype: int64
0    1.000000
2    2.228916
1    3.032787
Name: labels, dtype: float64
[1.0, 3.0327868852459017, 2.2289156626506026]


In [10]:
model = ClassificationModel(model_type, model_name, num_labels=3, weight=dist, args=model_args, use_cuda=True)
#model = ClassificationModel(model_type, model_name, num_labels=3, args=model_args, use_cuda=True)

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight', 'cl

In [11]:
model.train_model(train_df, eval_df=test_df)

c:\tesla-news-classifier\env\lib\site-packages\sklearn\metrics\_classification.py:873: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


(120,
 {'global_step': [6,
   12,
   18,
   24,
   30,
   36,
   42,
   48,
   54,
   60,
   66,
   72,
   78,
   84,
   90,
   96,
   102,
   108,
   114,
   120],
  'mcc': [0.0,
   0.11932546023467804,
   0.15592813435158998,
   0.19502651825533468,
   0.3159395904945592,
   0.2129618466686389,
   0.283453399126733,
   0.4084405891725789,
   0.3535937555613629,
   0.37991768009164767,
   0.3437805457912996,
   0.35236870004311144,
   0.3811633971594295,
   0.34075257018846333,
   0.38066525334393025,
   0.3856230600013103,
   0.4197452276952151,
   0.4073133808833592,
   0.353425403688454,
   0.3692528225225707],
  'train_loss': [1.1216851472854614,
   1.1244020462036133,
   0.9625990390777588,
   0.6936809420585632,
   0.6977770924568176,
   0.7195158004760742,
   0.5304768681526184,
   0.4457680583000183,
   0.5402554869651794,
   0.3546225428581238,
   0.4082144498825073,
   0.30879664421081543,
   0.44000324606895447,
   0.10232408344745636,
   0.13970057666301727,
   0.113151125

In [8]:
preds, out = model.predict(["consumer reports says it can't recommend the tesla model 3"])
#preds, out = model.predict(["tesla denies safety problems with model s suspensions"])
#preds, out = model.predict(["us ends probe of tesla fatal crash without seeking recall"])
print(preds)
print(out)


[1]
[[-0.18212891  0.50195312  0.26928711]]


In [ ]:
train_df.text.str.split(' ').str.len()